## Basic usage of Prompt

In [31]:
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# load env variables
load_dotenv()

llm = OpenAI(temperature=0)
chat_model = ChatOpenAI(temperature=0)

## predict
llm.predict("hi!")
chat_model.predict("hi!")

## predict_messages
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
llm.predict_messages(messages)
chat_model.predict_messages(messages)

AIMessage(content='VibrantSock Co.')

In [22]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)
prompt.format(product="colorful socks")

# It sames as:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [32]:
from langchain.prompts.chat import ChatPromptTemplate

template = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human_template = "{text}"

# Use one of 'human', 'user', 'ai', 'assistant', or 'system'
chat_prompt = ChatPromptTemplate.from_messages(
    [("system", template), ("human", human_template)]
)
chat_prompt.format_messages(
    input_language="English", output_language="French", text="I love programming."
)
from langchain.prompts.chat import ChatPromptTemplate

template = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human_template = "{text}"

# Use one of 'human', 'user', 'ai', 'assistant', or 'system'
chat_prompt = ChatPromptTemplate.from_messages(
    [("system", template), ("human", human_template)]
)
output = chat_prompt.format(
    input_language="English", output_language="French", text="I love programming."
)
# or alternatively
output_as_ChatPromptValue = chat_prompt.format_prompt(
    input_language="English", output_language="French", text="I love programming."
)
# format_prompt -> ChatPromptValue
assert output == output_as_ChatPromptValue.to_string()

output_as_messages = chat_prompt.format_prompt(
    input_language="English", output_language="French", text="I love programming."
).to_messages()
print(output_as_messages)


# see how the llm get inputs
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt, verbose=True)
chain.run(
    input_language="English", output_language="French", text="I love programming."
)

[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that translates English to French.
Human: I love programming.

> Finished chain.


"J'adore la programmation."

In [8]:
type(chat_prompt)

langchain.prompts.chat.ChatPromptTemplate

### Deep dive into PromptTemplate

In [2]:
from langchain.prompts import PromptTemplate, BasePromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke.")
print(prompt_template.format())
print("---------------------")
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
print(prompt_template.format(adjective="funny", content="chickens"))
print(
    f"""Parameter:
      {prompt_template.lc_attributes}
      {prompt_template.input_variables}
      {prompt_template.template}
      {prompt_template.template_format}
      {prompt_template.validate_template}
      {prompt_template._prompt_type}"""
)

print("---------------------")
## Specify input_variables explicitly.

# invalid_prompt = PromptTemplate(
#     input_variables=["adjective"],
#     template="Tell me a {adjective} joke about {content}."
# ) -> Validation Error
## You can disable the validation by adding:
## PromptTemplate(template=template, input_variables=["adjective"], validate_template=False)


## Custom PromptTemplate
## !pip3 install pydantic==1.10.12
## About the issue raised: https://github.com/langchain-ai/langchain/issues/9702

# Two requirements:
# 1. input_variables that the custom PromptTemplate expects
# 2. format() method that takes in the keyword arguments corresponding to the expected input_variables
# and returns the formatted prompt.

# Two distinct prompt templates:
# 1. StringPromptTemplate
# 2. ChatPromptTemplate

import inspect


def get_source_code(function_name):
    return inspect.getsource(function_name)


def sample_function(a, b):
    return a % b


print(get_source_code(sample_function))

from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, Field, validator

PROMPT = """\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    input_variables: list = Field(
        ..., description="The input variables for the prompt template"
    )

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"


fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)

Tell me a joke.
---------------------
Tell me a funny joke about chickens.
Parameter:
      {'template_format': 'f-string'}
      ['adjective', 'content']
      Tell me a {adjective} joke about {content}.
      f-string
      False
      prompt
---------------------
def sample_function(a, b):
    return a%b

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code
Source Code:
def get_source_code(function_name):
    return inspect.getsource(function_name)

Explanation:



### ChatPromptTemplate

In [6]:
from langchain.prompts import ChatPromptTemplate

# ChatPromptTemplate accepts a variety of message representations.
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
# Instead of 2-tuple representation, pass an instance of MessagePromptTemplate or BaseMessage.
from langchain.prompts import HumanMessagePromptTemplate  # MessagePromptTemplate
from langchain.schema.messages import SystemMessage  # BaseMessage
from langchain.chat_models import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant that re-writes the user's text to sound more upbeat."
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI()
llm(chat_template.format_messages(text="i dont like eating tasty things."))

AIMessage(content='I absolutely love indulging in delicious treats!')

## `PromptTemplate` & `ChatPromptTemplate`
- Implement Runnable interface
- Runnable interface: the basic block of the LangChain Expression Language (LCEL)
- it supports invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

`PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`. A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [19]:
from langchain.prompts import PromptTemplate

# PromptTemplate
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
# StringPromptValue(text='Tell me a funny joke about chickens.')
print(prompt_val.to_string())
print(prompt_val.to_messages())

from langchain.prompts.chat import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant that re-writes the user's text to sound more upbeat."
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
print(chat_template.to_json())

chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
# ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])
print(chat_val.to_messages())
print(chat_val.to_string())

Tell me a funny joke about chickens.
[HumanMessage(content='Tell me a funny joke about chickens.')]
{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'], 'kwargs': {'input_variables': ['text'], 'messages': [SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]}}
[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')]
System: You are a helpful assistant that re-writes the user's text to sound more upbeat.
Human: i dont like eating tasty things.


In [48]:
# MessagePromptTemplate
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
)

# However, in cases where the chat model supports taking chat message with arbitrary role,
# you can use ChatMessagePromptTemplate, which allows user to specify the role name.
from langchain.prompts import ChatMessagePromptTemplate

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)
chat_message = chat_message_prompt.format(subject="force")
print(chat_message)

chat_prompt = ChatPromptTemplate.from_messages(
    [chat_message, HumanMessagePromptTemplate.from_template("{text}")]
)
print(chat_prompt.format(text="Hi"))
print("------------")

# LangChain also provides MessagesPlaceholder,
# which gives you full control of what messages to be rendered during formatting.
# This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.
from langchain.prompts import MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage, SystemMessage

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

print(
    chat_prompt.format_prompt(
        conversation=[human_message, ai_message], word_count="10"
    ).to_string()
)

content='May the force be with you' role='Jedi'
Jedi: May the force be with you
Human: Hi
------------
Human: What is the best way to learn programming?
AI: 1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience
Human: Summarize our conversation so far in 10 words.


In [61]:
## Partial prompt template

# Partial formatting with string values.

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="{foo} {bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="BAR"))
# OR
prompt = PromptTemplate(template="{foo} {bar}", input_variables=["bar", "foo"])
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="BAR"))

# Partial formatting with functions that return string values.
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))
# OR
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

foo BAR
foo BAR
Tell me a funny joke about the day 10/28/2023 23:09:21
Tell me a funny joke about the day 10/28/2023 23:09:21


In [63]:
from langchain.prompts import PipelinePromptTemplate

## PipelinePromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)
print(pipeline_prompt.input_variables)
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

['example_a', 'person', 'example_q', 'input']
You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:
